# Use the langgraph-sdk to interact with your agent's API

In [46]:
%pip install langgraph_sdk
from langgraph_sdk import get_client

Note: you may need to restart the kernel to use updated packages.


In [47]:
client = get_client()

In [48]:
assistants = await client.assistants.search()
assistants

[{'assistant_id': 'f6142c1e-6175-4a9f-8297-5aca82d0819e',
  'graph_id': 'agent',
  'config': {},
  'created_at': '2024-05-17T17:59:00.499064+00:00',
  'updated_at': '2024-05-17T17:59:00.499064+00:00',
  'metadata': {}},
 {'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'graph_id': 'agent',
  'config': {},
  'created_at': '2024-05-17T01:11:16.548057+00:00',
  'updated_at': '2024-05-17T01:11:16.548057+00:00',
  'metadata': {'created_by': 'system'}}]

In [49]:
assistant = next(a for a in assistants if a['graph_id'] == 'agent')
assistant

{'assistant_id': 'f6142c1e-6175-4a9f-8297-5aca82d0819e',
 'graph_id': 'agent',
 'config': {},
 'created_at': '2024-05-17T17:59:00.499064+00:00',
 'updated_at': '2024-05-17T17:59:00.499064+00:00',
 'metadata': {}}

In [50]:
thread = await client.threads.create()
thread

{'thread_id': '8c683218-44c6-4b28-969f-9dcc9c7392fe',
 'created_at': '2024-05-17T22:11:40.209336+00:00',
 'updated_at': '2024-05-17T22:11:40.209336+00:00',
 'metadata': {}}

In [51]:
runs = await client.runs.list(thread['thread_id'])
runs

[]

In [52]:
input = {"messages": [{"role": "human", "content": "whats the weather in la"}]}
async for chunk in client.runs.stream(thread['thread_id'], assistant['assistant_id'], input=input):
    print(chunk)

StreamPart(event='metadata', data={'run_id': 'd5fa1da1-ea98-48e2-af18-0bcb17a703ae'})
StreamPart(event='error', data={'status_code': 400, 'message': 'Internal Server Error'})
StreamPart(event='end', data=None)


In [45]:
input = {"messages": [{"role": "human", "content": "and in sf"}]}
run = await client.runs.create(thread['thread_id'], assistant["assistant_id"], input=input)
run

Error from langgraph-api: {"detail":"There is already a pending run for this thread."}
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langgraph_sdk/client.py", line 69, in post
    r.raise_for_status()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/httpx/_models.py", line 758, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '409 Conflict' for url 'http://localhost:8123/threads/48d4109f-89e9-4b8b-a0b1-97b4f0dc9208/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409
{"detail":"There is already a pending run for this thread."}


HTTPStatusError: Client error '409 Conflict' for url 'http://localhost:8123/threads/48d4109f-89e9-4b8b-a0b1-97b4f0dc9208/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409

In [31]:
await client.runs.get(thread['thread_id'], run['run_id'])

{'run_id': 'd932feca-a174-45b3-9290-1714b5c9f7f3',
 'thread_id': '8b1c644e-48cc-4b07-9214-4363bca3592b',
 'assistant_id': 'f6142c1e-6175-4a9f-8297-5aca82d0819e',
 'created_at': '2024-05-17T22:03:56.664799+00:00',
 'updated_at': '2024-05-17T22:03:56.664799+00:00',
 'status': 'success',
 'metadata': {}}

In [32]:
await client.runs.list_events(thread['thread_id'], run['run_id'])

[{'event_id': 'cb6fbe92-d74e-4aa8-89d5-89eb5e8050b8',
  'run_id': 'd932feca-a174-45b3-9290-1714b5c9f7f3',
  'received_at': '2024-05-17T22:03:57.066912+00:00',
  'span_id': 'e2264367-8cff-405e-992d-d0c7fbdfc57d',
  'event': 'on_chain_end',
  'name': 'agent',
  'data': {'input': {'messages': [{'id': '4f059b93-a35d-4135-9415-1df0931e6c5a',
      'name': None,
      'type': 'human',
      'content': 'whats the weather in la',
      'example': False,
      'additional_kwargs': {},
      'response_metadata': {}},
     {'id': 'd8582dcd-084d-4718-8016-b5ddc2e39052',
      'name': None,
      'type': 'human',
      'content': 'and in sf',
      'example': False,
      'additional_kwargs': {},
      'response_metadata': {}}]},
   'output': None},
  'metadata': {'graph_id': 'agent',
   'thread_id': '8b1c644e-48cc-4b07-9214-4363bca3592b',
   'assistant_id': 'f6142c1e-6175-4a9f-8297-5aca82d0819e'},
  'tags': ['graph:step:3']},
 {'event_id': 'fd7434f7-7f9f-4f67-8c75-3d741cb6e278',
  'run_id': 'd932f